In [1]:
import os
import pandas as pd
from tqdm import tqdm
from llama_cpp import Llama
import base64

# --- 1. Конфигурация ---
# Убедитесь, что этот путь правильный!
MODEL_PATH = "./models/llava-v1.6-vicuna-13b.Q4_K_M.gguf" 
IMG_DIR = "img"
RESULTS_FILE = "llava_local_13b_results.csv"
PROMPT = "Скажи, изображена ли на данном фото кошка породы \"Британская короткошерстная\", дай коротки ответ \"да\" или \"нет\". Опиши одним словом каждый признак, который убедил тебя в правильности или неправильности ответа. Если это не британская короткошерстная кошка, скажи какая это порода."

# --- Настройка GPU Offloading ---
# Это самый важный параметр для вашей RTX 4060 (8 ГБ VRAM).
# Он определяет, сколько слоев модели выгрузить в видеопамять.
# 13B-модель (~8 ГБ) не поместится целиком.
# Хорошее начальное значение - 35. Если будут ошибки "out of memory", уменьшите до 30.
# Если VRAM будет использоваться не полностью, можно увеличить до 38.
N_GPU_LAYERS = 35

# --- 2. Загрузка модели ---
llm = None
# Проверяем, существует ли файл модели, прежде чем пытаться его загрузить
if os.path.exists(MODEL_PATH):
    print("Загрузка модели... Это может занять несколько минут, пока слои выгружаются на GPU.")
    try:
        llm = Llama(
            model_path=MODEL_PATH,
            n_ctx=2048,  # Размер контекста (окна внимания)
            n_gpu_layers=N_GPU_LAYERS, # <-- Применяем нашу настройку
            verbose=False # Отключаем лишний технический вывод
        )
        print("✅ Модель успешно загружена!")
    except Exception as e:
        print(f"❌ Ошибка загрузки модели: {e}")
else:
    print(f"❌ Файл модели не найден по пути: {MODEL_PATH}")
    print("Пожалуйста, убедитесь, что вы скачали модель и поместили ее в папку /models.")


# --- Вспомогательная функция для кодирования изображения ---
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# --- 3. Основной цикл обработки ---
if llm:
    try:
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        
        print(f"Начинаем обработку {len(image_files)} изображений локально на вашем GPU...")
        for filename in tqdm(image_files, desc="LLaVA Local 13B"):
            image_path = os.path.join(IMG_DIR, filename)
            
            try:
                base64_image = encode_image(image_path)
                
                # Создаем запрос в формате, который понимает LLaVA через llama-cpp-python
                response = llm.create_chat_completion(
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                                {"type": "text", "text": PROMPT}
                            ]
                        }
                    ]
                )
                
                result_text = response['choices'][0]['message']['content'].strip()
                results_data.append({"filename": filename, "llava_13b_response": result_text})

            except Exception as e:
                # Если одна картинка вызовет ошибку, мы запишем ее и продолжим
                results_data.append({"filename": filename, "llava_13b_response": f"Error: {e}"})
        
        df_results = pd.DataFrame(results_data)
        df_results.to_csv(RESULTS_FILE, index=False, encoding='utf-8')
        print(f"\n✅ Результаты сохранены в '{RESULTS_FILE}'.")
        display(df_results.head())

    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("\n❌ Основной процесс не был запущен, так как модель не была загружена.")

RuntimeError: Failed to load shared library 'c:\Users\basal\AppData\Local\Programs\Python\Python312\Lib\site-packages\llama_cpp\lib\llama.dll': Could not find module 'c:\Users\basal\AppData\Local\Programs\Python\Python312\Lib\site-packages\llama_cpp\lib\llama.dll' (or one of its dependencies). Try using the full path with constructor syntax.